In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

In [2]:
# Data gathering
df = pd.read_csv("data2.TXT", sep=";")
df = df.drop(["Temp1-18B20 (°C)", "Conc. gas-MQ135 (ppm)"], axis=1)
df["Data/Hora"] = pd.to_datetime(df["Data/Hora"])
df.index = df["Data/Hora"]


# The data I'm using seems to be consistent only between 05-13-2021 and 05-31-2021
# Filter days before
def FiltrarDiaB(df, dia, mes, ano):
    return df.loc[df.index <= pd.Timestamp("{}-{}-{}".format(ano, mes, dia))]

# Filter days after
def FiltrarDiaA(df, dia, mes, ano):
    return df.loc[df.index >= pd.Timestamp("{}-{}-{}".format(ano, mes, dia))]

df = FiltrarDiaA(df, 13, 5, 2021)
df = FiltrarDiaB(df, 31, 5, 2021)

df

,Data/Hora,Temp2-18B20 (°C),Temp3-18B20 (°C),Temp4-18B20 (°C),Temp5-18B20 (°C),Temp6-18B20 (°C),Temp1-dht22 (°C),Umid1-dht22 (%),Temp2-dht22 (°C),Umid2-dht22 (%),Temp3-dht22 (°C),Umid3-dht22 (%),Corrente1 (A),Corrente2 (A),N° de pessoas,Luminosidade (lux),Conc. CO2 (ppm)
Data/Hora,,,,,,,,,,,,,,,,,
2021-05-13 00:00:00,2021-05-13 00:00:00,22.7,22.7,23.0,23.1,22.6,22.7,86.6,26.6,65.0,17.9,99.9,0.0,0.0,10,0.0,670
2021-05-13 00:30:00,2021-05-13 00:30:00,22.6,22.6,23.0,22.9,22.6,22.7,87.1,26.8,65.0,17.9,99.9,0.0,0.0,10,0.0,671
2021-05-13 01:00:00,2021-05-13 01:00:00,22.5,22.6,22.9,22.9,22.5,22.6,87.2,26.6,65.0,17.7,99.9,0.0,0.0,10,0.0,667
2021-05-13 01:30:00,2021-05-13 01:30:00,22.4,22.6,22.9,22.9,22.4,22.6,87.4,26.6,65.0,17.9,99.9,0.0,1.1,10,0.0,670
2021-05-13 02:00:00,2021-05-13 02:00:00,22.4,22.5,22.9,22.9,22.4,22.5,87.5,26.6,66.0,17.9,99.9,0.0,0.0,10,0.0,669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-30 22:58:00,2021-05-30 22:58:00,23.3,23.3,23.6,23.8,23.1,23.1,93.6,27.1,67.0,19.5,99.9,1.2,0.0,12,0.0,677
2021-05-30 22:59:00,2021-05-30 22:59:00,23.2,23.3,23.6,23.8,23.1,23.1,93.6,27.1,67.0,19.5,99.9,0.0,0.0,12,0.0,678
2021-05-30 23:00:00,2021-05-30 23:00:00,23.2,23.3,23.5,23.8,23.1,23.1,93.5,27.1,67.0,19.5,99.9,0.0,0.0,12,0.0,680


In [3]:
# App inicialization
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [4]:
# App Components
Header1 = html.H1('Dash Tabs component demo', 
                  style={'textAlign': 'center'}) # App title

# Tabs of the tab component
TabTemperature = dcc.Tab(label='Temperature (ºC)', value='TabTemperature')
TabLuminosity = dcc.Tab(label='Luminosity (Lux)', value='TabLuminosity')
TabCO2 = dcc.Tab(label='CO2 (ppm)', value='TabCO2')
TabHumity = dcc.Tab(label='Humidity (%)', value='TabHumity')
TabAC = dcc.Tab(label='AC Use (A)', value='TabAC')

# Tabs component
Tabs = dcc.Tabs(id="tabs-example-graph", 
                value='TabTemperature', 
                children=[TabTemperature, TabLuminosity, TabCO2, TabHumity, TabAC])

# Graph placeholder for tabs
placeholder = html.Div(id='tabs-content-graph')

# Slider Widget
dateRange = list(range(min(df.index.day), max(df.index.day), 3))
dates = df.index.strftime('%m/%d/%Y').unique()
marks = dict(zip(dateRange, dates))

yearSlider = dcc.RangeSlider(min(df.index.day), max(df.index.day), 1,
                             marks=marks,
                             value=[min(df.index.day), max(df.index.day)], 
                             id='year-range-slider')


msTemp = dcc.Dropdown(id='temp-select',
                      options=[{'label': 'Door', 'value': 'Temp3-18B20 (°C)'},
                              {'label': 'Window', 'value': 'Temp4-18B20 (°C)'},
                              {'label': 'Center', 'value': 'Temp2-18B20 (°C)'}],
                      value=["Temp3-18B20 (°C)"],
                      multi=True,
                      style={'width': '50%','padding': 10})


msHum = dcc.Dropdown(id='hum-select',
                     options=[{'label': 'Door', 'value': 'Umid1-dht22 (%)'},
                              {'label': 'Window', 'value': 'Umid2-dht22 (%)'},
                              {'label': 'Center', 'value': 'Umid3-dht22 (%)'}],
                     value=["Umid1-dht22 (%)"],
                     multi=True,
                     style={'width': '50%','padding': 10})

multiGraphs = html.Div(id='multi-graphs')

selectors = html.Div(id='selectors',
                    children=[msTemp, msHum],
                    style={'display': 'flex'})


# Define app layout
app.layout = html.Div([Header1, Tabs, placeholder, yearSlider, selectors, multiGraphs])

In [5]:
# Tabs callback
@app.callback(Output('tabs-content-graph', 'children'),
              [Input('tabs-example-graph', 'value'),
              Input('year-range-slider', 'value')])
def render_content(tab, slider_value):
    # Dictionary {keys -> tab name      values -> y axis values}
    yAxis = {'TabTemperature': "Temp2-18B20 (°C)",
            'TabLuminosity': "Luminosidade (lux)",
            'TabCO2': "Conc. CO2 (ppm)",
            'TabHumity': "Umid2-dht22 (%)",
            'TabAC': ["Corrente1 (A)", "Corrente2 (A)"]}
    
    # Data Frame filtered by date slider
    df2 = FiltrarDiaA(df, slider_value[0], 5, 2021)
    df2 = FiltrarDiaB(df2, slider_value[1], 5, 2021)
    
    fig = px.line(df2, y=yAxis[tab], x="Data/Hora") # Graph made by pyplot
    
    TabContent = html.Div([html.H3('Temperature', 
                                       style={'textAlign': 'center'}), # A title
                               dcc.Graph(figure=fig)]) # The graph
    
    return TabContent



@app.callback(Output('multi-graphs', 'children'),
             [Input('temp-select', 'value'),
              Input('hum-select', 'value'),
              Input('year-range-slider', 'value')])
def render_temp_graph(selTemp, selHum, slider_value):
    
     # Data Frame filtered by date slider
    df2 = FiltrarDiaA(df, slider_value[0], 5, 2021)
    df2 = FiltrarDiaB(df2, slider_value[1], 5, 2021)
    
    figTemp = px.line(df2, y=selTemp, x="Data/Hora")
    figHum = px.line(df2, y=selHum, x="Data/Hora")
    
    content = [dcc.Graph(figure=figTemp, style={'display': 'inline-block'}), 
               dcc.Graph(figure=figHum, style={'display': 'inline-block'})]
    
    return content

In [6]:
app.run_server()

Dash app running on http://127.0.0.1:8050/
